In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# OpenAI API 키 설정
import openai

# ChromaDB 경로 설정
db_path = "C:/Users/user/Desktop/langchain-kr/hk-project/chromaDB"  # 저장된 경로

# 임베딩 함수 설정 (저장 당시와 동일한 임베딩 함수 사용)
embeddings = OpenAIEmbeddings()

# ChromaDB 로드
vectorstore = Chroma(persist_directory=db_path, embedding_function=embeddings)

print("ChromaDB 로드 완료.")

ChromaDB 로드 완료.


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# 사용자 소비내역
query = "식당: 80000, 카페: 45000, 편의점: 20000"

# Retrieval
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)

# Prompt 설정
prompt_template = ChatPromptTemplate.from_template(
"""
사용자 소비 내역은 다음과 같습니다:
{question}

위 소비 내역에 대해, 아래 카드 데이터 중에서 가장 적합한 카드 3개를 추천해주세요:
{context}

각 카드에 대해 소비 내역과 연결된 구체적인 혜택과 이유를 간단히 설명해주세요.
혜택의 할인 한도를 초과하지 않도록 추천하세요.
혜택 할인 한도를 알 수 없는 경우는 정확한 할인 금액은 제시하지 마세요.
"""
)

# Model 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

# Chain 생성
chain = LLMChain(llm=llm, prompt=prompt_template)

# 문서 검색 및 포맷팅
docs = retriever.get_relevant_documents(query)
context = "\n\n".join([doc.page_content for doc in docs])

# 실행
response = chain.run({"context": context, "question": query})
print(response)


각 카드의 혜택을 사용자 소비 내역과 연결하여 추천해 드리겠습니다.

### 1. 삼성 iD MOVE 카드
**연회비:** 국내 2만원, 해외 2만원  
**혜택:** 
- 카페 10% 결제일 할인 (스타벅스, 이디야커피 등)
- 편의점 10% 결제일 할인 (CU, GS25 등)

**소비 내역과의 연결:** 
- 카페에서 45,000원을 소비하였으므로, 10% 할인 혜택을 통해 4,500원의 할인을 받을 수 있습니다.
- 편의점에서 20,000원을 소비하였으므로, 10% 할인 혜택을 통해 2,000원의 할인을 받을 수 있습니다.
- 총 할인 금액: 6,500원.

이 카드의 연회비가 낮고, 카페와 편의점 이용에 적합한 혜택이 있어 추천합니다.

### 2. D4카드의정석Ⅱ
**연회비:** 국내 1만2천원, 해외 1만2천원  
**혜택:**
- 카페 55% 청구할인 (스타벅스, 투썸플레이스 등)
- 편의점 11% 청구할인
- 영화관 청구할인

**소비 내역과의 연결:** 
- 카페에서 45,000원을 소비하였으므로, 55% 할인 혜택을 통해 24,750원의 할인을 받을 수 있습니다.
- 편의점에서 20,000원을 소비하였으므로, 11% 할인 혜택을 통해 2,200원의 할인을 받을 수 있습니다.
- 총 할인 금액: 26,950원.

이 카드의 카페 할인 혜택이 매우 크기 때문에 추천합니다.

### 3. 디지로카 Monaco 카드
**연회비:** 국내 12만5천원, 해외 12만5천원  
**혜택:** 
- 카페/베이커리 1% 할인
- 편의점 1% 할인
- 외식 1% 할인

**소비 내역과의 연결:** 
- 카페에서 45,000원을 소비하였으므로, 1% 할인 혜택을 통해 450원의 할인을 받을 수 있습니다.
- 편의점에서 20,000원을 소비하였으므로, 1% 할인 혜택을 통해 200원의 할인을 받을 수 있습니다.
- 식당에서 80,000원을 소비하였으므로, 1% 할인 혜택을 통해 800원의 할인을 받을 수 있습니다.
- 총 할인 금액: 1,450원.

이 카드는 

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# 사용자 소비내역
query = "식당: 80000, 카페: 45000, 편의점: 20000, 주유: 130000, 온라인쇼핑: 200000, 영화: 14000"

# Retrieval
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.5}
)

# Prompt 설정
prompt_template = ChatPromptTemplate.from_template(
    """
    시스템 역할:
    당신은 카드 추천 매니저입니다.
    아래의 지시사항을 따르세요:
    - 각 카드에 대해 소비 내역과 연결된 구체적인 혜택과 이유를 간단히 설명하세요.
    - 혜택의 할인 한도를 초과하지 않도록 추천하세요.
    - 혜택 할인 한도를 알 수 없는 경우는 정확한 할인 금액은 제시하지 마세요.

    예시 형식:
        - 카드명 : 현대카드M
        - 연회비 : 국내 3만원, 해외 3만원, 가족 1만원
        - 기준 실적 : 직전 1개월 합계 50만원 이상
        혜택
        - 혜택 범위 : 주유, 통신, 대중교통 등
        - 혜택 상세 : 혜택 요약된 결과

        - 카드명 : 신한카드 처음
        - 연회비 : 국내 1만5천원, 해외 1만8천원
        - 기준 실적 : 직전 1개월 합계 30만원 이상
        혜택
        - 혜택 범위 : 통신, 외식, 편의점 등
        - 혜택 상세 : 혜택 요약된 결과

        - 카드명 : LOCA Professional
        - 연회비 : 국내 30만원, 해외 30만원, 가족 5만원
        - 기준 실적 : 조건없음
        혜택
        - 혜택 범위 : 영화, 쇼핑, 대형마트 등
        - 혜택 상세 : 혜택 요약된 결과

    유저 역할:
    사용자 소비 내역:
    {question}

    카드 데이터:
    {context}

    위 데이터를 기반으로 가장 적합한 카드 3개를 추천하고, 각 카드의 설명을 위의 형식에 맞게 작성하세요.
    """
)

# Model 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

# RetrievalQA 체인 생성
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt_template, "document_variable_name": "context"}
)

# 실행
response = rag_chain({"query": query})

# 결과 출력
print("추천 카드:")
print(response["result"])

# 참고 문서도 보고 싶으면 return_source_documents=True로 주고 아래 코드 활성화
# print("\n참고 문서:")
# for doc in response["source_documents"]:
#     print(doc.page_content)


추천 카드:
- 카드명 : 디지로카 Monaco
  - 연회비 : 국내 12만5천원, 해외 12만5천원, 가족 2만원
  - 기준 실적 : 직전 1개월 합계 1백만원 이상
  혜택
  - 혜택 범위 : 주유, 통신, 외식, 온라인 쇼핑, 영화 등
  - 혜택 상세 : 주유비 1% 할인, 온라인 쇼핑 이용 시 2% 결제일 할인. 영화 1% 할인, 편의점 및 카페 1% 할인 제공. 주유 리터당 70원 할인 가능 (50만원 이상 실적 시).

- 카드명 : 아메리칸 익스프레스 블루 KB국민카드
  - 연회비 : 국내 1만9천원, 해외 1만9천원
  - 기준 실적 : 직전 1개월 합계 30만원 이상
  혜택
  - 혜택 범위 : 온라인 쇼핑, 주유, 영화, 카페 등
  - 혜택 상세 : G마켓 등 온라인쇼핑 10% 할인, 주유 시 100원/L당 할인, 영화 티켓 5,000원 결제일 할인. 카페 아티제에서 커피 1+1 제공 서비스.

- 카드명 : 삼성 iD AUTO 카드
  - 연회비 : 국내 4만9천원, 해외 4만9천원
  - 기준 실적 : 직전 1개월 합계 50만원 이상
  혜택
  - 혜택 범위 : 주유, 편의점, 영화, 대형마트 등
  - 혜택 상세 : 주유 시 리터당 70원 할인 (50만원 이상 실적 시), 영화 티켓 5,000원 결제일 할인, 편의점 1.5% 결제일 할인. 대형마트 및 온라인쇼핑몰에서도 1.5% 할인 제공.
